<a href="https://colab.research.google.com/github/airas-org/airas/blob/develop/airas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install --index-url https://test.pypi.org/simple/ \
            --extra-index-url https://pypi.org/simple \
            airas==0.0.11.dev56

In [2]:
import logging

log_format = "[%(levelname)s] %(name)s: %(message)s"
logging.basicConfig(level=logging.INFO, format=log_format, force=True)

In [3]:
import os

from google.colab import userdata

# ==============================================================================
# シークレットキーの設定
# ==============================================================================
# 手順:
# 1. Google Colab のサイドバーにある「シークレット」マネージャー（鍵アイコン）を開きます。
# 2. 以下にリストされている各シークレット名（例: "OPENAI_API_KEY"）を追加し、
#    対応する値を貼り付けます。
# ==============================================================================


# --- 必須APIキー ---

# OpenAI APIキー
# URL: https://platform.openai.com/settings/organization/api-keys
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

# GitHub パーソナルアクセストークン  (Fine-grained)
# URL: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
#
# トークンに必要な設定:
# - リソースオーナー: "auto-res2"
# - 権限:    すべてのスコープ（リポジトリ、ワークフローなど）にフルアクセスを付与します。
os.environ["GITHUB_PERSONAL_ACCESS_TOKEN"] = userdata.get(
    "GITHUB_PERSONAL_ACCESS_TOKEN"
)


# --- 追加APIキー ---

# os.environ["GEMINI_API_KEY"] = userdata.get("GEMINI_API_KEY")
# os.environ["DEVIN_API_KEY"] = userdata.get("DEVIN_API_KEY")
# os.environ["QDRANT_API_KEY"] = userdata.get("QDRANT_API_KEY")
# os.environ["SEMANTIC_SCHOLAR_API_KEY"] = userdata.get("SEMANTIC_SCHOLAR_API_KEY")

In [4]:
from airas.features import (
    AnalyticSubgraph,
    CreateBibfileSubgraph,
    CreateBranchSubgraph,
    CreateCodeSubgraph,
    CreateExperimentalDesignSubgraph,
    CreateMethodSubgraph,
    ExtractReferenceTitlesSubgraph,
    FixCodeSubgraph,
    FixCodeWithDevinSubgraph,
    GenerateQueriesSubgraph,
    GetPaperTitlesFromDBSubgraph,
    GitHubActionsExecutorSubgraph,
    GithubDownloadSubgraph,
    GithubUploadSubgraph,
    HtmlSubgraph,
    LatexSubgraph,
    PrepareRepositorySubgraph,
    ReadmeSubgraph,
    RetrieveCodeSubgraph,
    RetrievePaperContentSubgraph,
    ReviewPaperSubgraph,
    SummarizePaperSubgraph,
    WriterSubgraph,
)

In [5]:
# ==============================================================================
# 設定: 研究タスクを定義
# ==============================================================================
# これはユーザー入力の主要なセルです。
# 以下の変数を変更して、リポジトリと研究目標を設定します。
# ==============================================================================


# --- GitHubリポジトリ設定 ---

# リポジトリのオーナーを設定します。
github_owner = "your-owner-name"

# リポジトリ名を設定します。
repository_name = "your-repository-name"

# ブランチ名を設定します。
# 各ブランチは、研究トピックに対する個別の試みまたは実験を表します。
branch_name = "your-branch-name"


# --- 研究トピック ---

# ここに研究トピックを記述します。
# 例: "拡散モデルを高速化したいです。"
research_topic = "your-research-topic"

In [6]:
# ==============================================================================
# LLM選択
# ==============================================================================
# このセルは、サポートされているすべての大規模言語モデル (LLM) のリストを定義します。
# これらのリストにある任意のモデルは、どのサブグラフ内でも使用できます。
# ==============================================================================


# このセッションのデフォルトLLMモデルを設定します。
llm_name = "o3-2025-04-16"


# --- サポートされているモデル ---

OPENAI_MODEL = [
    "o4-mini-2025-04-16",
    "o3-2025-04-16",
    "o3-mini-2025-01-31",
    "o1-pro-2025-03-19",
    "o1-2024-12-17",
    "gpt-5-2025-08-07",
    "gpt-5-mini-2025-08-07",
    "gpt-5-nano-2025-08-07",
    "gpt-4.1-2025-04-14",
    "gpt-4o-2024-11-20",
    "gpt-4o-mini-2024-07-18",
]

# GEMINI_API_KEY が利用可能な場合
VERTEXAI_MODEL = [
    "gemini-2.5-pro",
    "gemini-2.5-flash",
    "gemini-2.5-flash-lite-preview-06-17",
    "gemini-2.0-flash-001",
    "gemini-2.0-flash-lite-001",
    "gemini-embedding-001",
]

In [7]:
# ==============================================================================
# サブグラフ設定
# ==============================================================================
# このセルでは、研究ワークフローに必要なすべてのサブグラフをインスタンス化します。
# 論理的なフェーズにグループ化して分かりやすくしています。
# 1. 研究と情報収集
# 2. 実装と実験
# 3. 執筆と出版

In [8]:
# --- フェーズ1: 研究と情報収集 ---


# 研究トピックから検索クエリを生成するサブグラフ。
# ここでは、設定した研究トピック（例：「拡散モデルにおける記憶」）を基に、
# AIが論文を検索するためのキーワードの組み合わせを `n_queries` 件自動で生成します。
generate_queries = GenerateQueriesSubgraph(llm_name=llm_name, n_queries=5)


# 生成したクエリでローカルデータベース（AIRASのリポジトリやQdrand）から論文タイトルを取得するためのサブグラフ。
# ここでは、意味が近い論文を検索し（semantic_search=True）、各クエリにつき最大 `max_results_per_query` 件の論文タイトルを取得します。
get_paper_titles = GetPaperTitlesFromDBSubgraph(
    max_results_per_query=3, semantic_search=False
)
# get_paper_titles = GetPaperTitlesFromWebSubgraph(max_results_per_query=5) # Webから直接検索する場合はこちらを有効化


# 検索した論文の全文を取得するサブグラフ。
# 取得した論文タイトルを基に、ArXiv（論文投稿サイト）から論文のPDFを探し、
# その内容（アブストラクト、本文、参考文献など）をテキストデータとして取得します。
retrieve_paper_content = RetrievePaperContentSubgraph(
    paper_provider="arxiv", target_study_list_source="research_study_list"
)


# 取得した論文内容をAIが要約するためのサブグラフ。
# 取得した各論文の全文をAIに入力し、
# 「主な貢献」「使われた技術」「実験設定」「限界」「今後の課題」といった項目で整理された要約を作成します。
summarize_paper = SummarizePaperSubgraph(llm_name=llm_name)


# 論文に関連するソースコードを取得するためのサブグラフ。
# 論文の本文中からGitHubリポジトリへのリンクを探し出します。
# リンクが見つかった場合、そのリポジトリにアクセスし、関連するPythonのソースコード（.pyファイル）を全て取得します。
retrieve_code = RetrieveCodeSubgraph(llm_name=llm_name)


# 論文の参考文献リストを抽出（孫引き調査の準備） するためのサブグラフ。
# 取得した論文の末尾にある「参考文献」セクションをAIが読み取り、
# 引用されている他の論文のタイトルを最大 `paper_retrieval_limit` 件までリストアップします。
reference_extractor = ExtractReferenceTitlesSubgraph(
    llm_name=llm_name, paper_retrieval_limit=10
)


# 参考文献として挙げられていた論文全文を取得するためのサブグラフ。
# リストアップされた参考文献のタイトルを使い、ArXivから論文の全文を取得します。
# これにより、元の論文がどのような研究に基づいているかをさらに深く調査できます。
retrieve_reference_paper_content = RetrievePaperContentSubgraph(
    paper_provider="arxiv", target_study_list_source="reference_research_study_list"
)

In [9]:
# --- フェーズ2: 実装と実験 ---


# 新しい研究手法を作成するためのサブグラフ。
# 収集した論文の要約やソースコードをAIが統合的に分析し、
# それらに基づいた新規研究アプローチや改善案を考案します。
# アイデアは `refine_iterations` 回にわたって自己改良され、より洗練された手法に仕上がります。
create_method = CreateMethodSubgraph(llm_name=llm_name, refine_iterations=5)


# 実験計画の設計するためのサブグラフ。
# 考案された新規手法の有効性を証明するための「実験計画」をAIが設計します。
# どのようなデータセットを使い、何を評価指標とし、どの既存手法と比較するのか、といった具体的な計画書を作成します。
create_experimental_design = CreateExperimentalDesignSubgraph(llm_name=llm_name)


# 実験用コードの自動生成するためのサブグラフ。
# 設計された実験計画書を基に、AIコーディングエージェント（Devinなど）が
# 実験を実行するためのPythonコードをゼロから書き上げます。
coder = CreateCodeSubgraph(llm_name=llm_name)
# coder = CreateCodeWithDevinSubgraph() # Devinを使用する場合はこちらを有効化


# GitHub Actions を使用して生成されたコードを実行するためのサブグラフ。
# 生成されたコードを、GPUが利用可能なGitHub Actionsの仮想環境上で自動的に実行します。
# GPUが利用可能な場合はgpu_enabled=True
# これにより、手元のPCに負荷をかけることなく、計算量の多い実験を行えます。実行結果（ログや生成物）が取得されます。
executor = GitHubActionsExecutorSubgraph(gpu_enabled=False)


# コードのエラーを自動修正するためのサブグラフ。
# コード実行中にエラーが発生した場合、このサブグラフが起動します。
# AIがエラーメッセージとソースコードを分析し、バグの原因を特定して自動でコードを修正します。
fixer = FixCodeSubgraph(llm_name=llm_name)
# fixer = FixCodeWithDevinSubgraph(llm_name=llm_name) # Devinを使用する場合はこちらを有効化


# 実験結果を分析するためのサブグラフ。
# 得られた実験結果（数値データやログ）をAIが分析します。
# 計画段階で設定した評価指標に基づいて、新しい手法が既存の手法と比べてどれだけ優れているかを評価し、結論を導き出します。
analysis = AnalyticSubgraph(llm_name=llm_name)

In [10]:
# --- フェーズ3: 執筆と出版 ---


# 参考文献リスト（BibTeX）の作成をするためのサブグラフ。
# 論文中で引用したすべての参考文献について、BibTeXという形式のデータベースファイルを作成します。
# ここでは、国際会議ICLRの2024年フォーマットに準拠し、引用文献は最大 `max_filtered_references` 件まで含めるように設定されています。
create_bibfile = CreateBibfileSubgraph(
    llm_name=llm_name, latex_template_name="iclr2024", max_filtered_references=30
)


# 論文の自動執筆するためのサブグラフ。
# これまでの全フェーズ（研究、実装、実験、分析）で得られた情報をAIがすべて統合し、
# 学術論文の形式（序論、関連研究、提案手法、実験、結論など）に沿って、論文の初稿を自動で執筆します。
# 執筆された内容は、AI自身によって `max_refinement_count` 回推敲され、品質が向上します。
writer = WriterSubgraph(llm_name=llm_name, max_refinement_count=2)


# AIによる論文の査読するためのサブグラフ。
# 執筆された論文を、AIが査読者（レビューア）の視点で厳しくチェックします。
# 「新規性」「明確さ」「技術的な正しさ」などの複数の評価基準に基づき、論文の弱点や改善点を指摘します。
review = ReviewPaperSubgraph(llm_name=llm_name)


# LaTeX形式の論文を生成するためのサブグラフ。
# 執筆された本文とBibTeXファイルを組み合わせ、学術界で標準的に使われる組版システム「LaTeX」を用いて、
# ICLR 2024の投稿規定に沿った美しいレイアウトのPDF論文を生成します。AIによる自動校正が `max_revision_count` 回行われます。
latex = LatexSubgraph(
    llm_name=llm_name, latex_template_name="iclr2024", max_revision_count=3
)


# 論文のHTML（ウェブページ）版を生成するためのサブグラフ。
# 生成された論文を、ウェブブラウザで手軽に閲覧できるHTML形式に変換します。
# このHTMLはGitHub Pagesとして公開することを想定しており、誰でも簡単に論文内容にアクセスできるようになります。
html = HtmlSubgraph(llm_name=llm_name)


# プロジェクトのREADMEファイルを作成するためのサブグラフ。
# この研究プロジェクト全体の概要をまとめた「README.md」ファイルを自動で生成します。
# プロジェクトの目的、使い方、これまでの成果などが記述され、他の人が見ても内容がすぐわかるようになります。
readme = ReadmeSubgraph()


# 成果物をGitHubにアップロードするためのサブグラフ。
# 生成された成果物（ソースコード、論文PDF、README、HTML版論文など）を、
# 一つのプロジェクトとしてGitHubリポジトリにアップロードし、保存できます。
uploader = GithubUploadSubgraph()

In [12]:
# ==============================================================================
# GITHUBリポジトリの準備
# ==============================================================================

from airas.types.github import GitHubRepositoryInfo

# GitHubリポジトリ情報と研究トピックを設定します。
state = {
    "github_repository_info": GitHubRepositoryInfo(
        github_owner=github_owner,
        repository_name=repository_name,
        branch_name=branch_name,
    ),
    "research_topic": research_topic,
}

# 全体の成果物を管理するGitHubリポジトリを作成
# 研究プロセス（論文調査、実装、執筆）の成果物を一元管理するために、
# https://github.com/airas-org/airas-template をテンプレートとして使用し、
# 新しいGitHubリポジトリを作成します。

_ = PrepareRepositorySubgraph().run(state)

In [13]:
# ==============================================================================
# 研究ワークフローの実行
# ==============================================================================

from tqdm import tqdm

subgraph_list = [
    generate_queries,
    get_paper_titles,
    retrieve_paper_content,
    summarize_paper,
    retrieve_code,
    create_method,
    create_experimental_design,
    coder,
    executor,
    fixer,
    analysis,
    reference_extractor,
    retrieve_reference_paper_content,
    create_bibfile,
    writer,
    review,
    latex,
    html,
    readme,
]


def run_subgraphs(subgraph_list, state):
    for subgraph in tqdm(subgraph_list, desc="Executing Research Workflow"):
        subgraph_name = subgraph.__class__.__name__
        print(f"--- Running Subgraph: {subgraph_name} ---")

        if isinstance(subgraph, (FixCodeSubgraph, FixCodeWithDevinSubgraph)):
            while True:
                if state.get("executed_flag") is True:
                    state = analysis.run(state)
                    break
                else:
                    state = fixer.run(state)
                    state = executor.run(state)
        else:
            state = subgraph.run(state)

        _ = uploader.run(state)
        print(f"--- Finished Subgraph: {subgraph_name} ---\n")


# 研究ワークフローを実行します。
_ = run_subgraphs(subgraph_list, state)

In [ ]:
# ==============================================================================
# オプション: 新しいブランチを作成し、サブグラフから再開
# ==============================================================================
# プロセスを途中で新しいブランチを作成し、特定の時点から再実行したい場合にこのセルを使用します。
#
# これは、ワークフローの途中で元のブランチを変更せずに、異なるアプローチを試すのに役立ちます。
# ==============================================================================


# --- 設定 ---

# 新しいブランチ名を設定します。
new_branch_name = "your-new-branch-name"

# 実行を再開したいサブグラフ名を設定します。
# 例: "CreateMethodSubgraph"
restart_from_subgraph = "subgraph-name-to-restart-from"


# 新規ブランチを作成します。
_ = CreateBranchSubgraph(
    new_branch_name=new_branch_name,
    restart_from_subgraph=restart_from_subgraph,
).run(state)

In [ ]:
# 特定のサブグラフからワークフローを再開するには（例: `restart_from_subgraph` 変数で定義されたサブグラフ）、
# このリストで、目的の開始点より前の行を手動でコメントアウトしてください。
#
# 例: `create_method` から再開するには、リストを次のように変更します。

subgraph_list = [
    # generate_queries,
    # get_paper_titles,
    # retrieve_paper_content,
    # summarize_paper,
    # # retrieve_code,
    create_method,
    create_experimental_design,
    coder,
    executor,
    fixer,
    analysis,
    reference_extractor,
    retrieve_reference_paper_content,
    create_bibfile,
    writer,
    review,
    latex,
    html,
    readme,
]


# GitHubリポジトリ情報を再設定します。
state = {
    "github_repository_info": GitHubRepositoryInfo(
        github_owner=github_owner,
        repository_name=repository_name,
        branch_name=new_branch_name,
    ),
}
# 新規ブランチから必要なデータをダウンロードします。
state = GithubDownloadSubgraph().run(state)

# 研究ワークフローを再実行します。
_ = run_subgraphs(subgraph_list, state)